# Kaggle SQL Analysis


We'll be using sqlite to access a database. File: https://www.kaggle.com/datasets/kaggle/sf-salaries. Check out the description of the data so you know the table / column names.

The following code will use sqlite to create a database connection.

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("database.sqlite")
crsr = conn.cursor()


In [2]:
df = pd.read_csv("Salaries.csv")
df.to_sql("salaries",conn, if_exists="replace", index=False)

C:\Users\dagem\AppData\Local\Temp\ipykernel_2028\4046911114.py:1: DtypeWarning: Columns (3,4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Salaries.csv")


148654

In [3]:
query = "SELECT * FROM salaries;"
full_table = pd.read_sql(query, conn)
full_table.head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,None,567595.43,567595.43,2011,None,San Francisco,None
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,None,538909.28,538909.28,2011,None,San Francisco,None
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,None,335279.91,335279.91,2011,None,San Francisco,None
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,None,332343.61,332343.61,2011,None,San Francisco,None
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,None,326373.19,326373.19,2011,None,San Francisco,None


# Exploration


Create a query that gives you a data frame of the EmployeeName, JobTitle, and BasePay from the salaries table. 

In [4]:
query = "SELECT EmployeeName,JobTitle,BasePay FROM salaries;"
df1 = pd.read_sql(query, conn)
df1.head()

,EmployeeName,JobTitle,BasePay
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6



Modify your query from Problem 1 to limit it to the year 2012. 

In [5]:
query = 'SELECT EmployeeName,JobTitle,BasePay, Year FROM salaries WHERE Year = 2012;'
df2 = pd.read_sql(query, conn)
df2.head()

,EmployeeName,JobTitle,BasePay,Year
0,Gary Altenberg,"Lieutenant, Fire Suppression",128808.87,2012
1,Gregory Suhr,Chief of Police,302578.0,2012
2,Khoa Trinh,Electronic Maintenance Tech,111921.0,2012
3,Joanne Hayes-White,"Chief, Fire Department",296943.01,2012
4,Frederick Binkley,EMT/Paramedic/Firefighter,126863.19,2012



Further limit the table to the year 2012, employees making under 150,000, and sort in descending order by salary. 

In [6]:
query = 'SELECT EmployeeName,JobTitle,BasePay,Year FROM salaries WHERE Year = 2012 AND BasePay < 150000 ORDER BY BasePay DESC;'
df3 = pd.read_sql(query, conn)
df3.head()

,EmployeeName,JobTitle,BasePay,Year
0,Aiko Yoshino,Recreation Leader,15000.37,2012
1,Sharita Williams,Public Svc Aide-Public Works,1500.06,2012
2,Kerry Phillips,Recreation Leader,150.41,2012
3,Stephanie Simmons,"BdComm Mbr, Grp2,M=$25/Mtg",150.0,2012
4,Ken Prag,Commissioner No Benefits,150.0,2012


# Aggregation

Get the average base pay from the table. Use `AverageBasePay` as the column name for the average base pay in your query result

In [7]:
query = 'SELECT AVG(BasePay) AS AverageBasePay FROM salaries;'
df4 = pd.read_sql(query, conn)
df4.head()

,AverageBasePay
0,66323.656854




Produce and print the head of a dataframe that shows the average pay for each year (only use a single, simple query).

In [8]:
query = 'SELECT Year, AVG(BasePay) AS AverageBasePay FROM salaries GROUP BY Year;'
df5 = pd.read_sql(query, conn)
print(df5)

   Year  AverageBasePay
0  2011    63595.956517
1  2012    65436.406857
2  2013    69630.030216
3  2014    66557.437750



Create a dataframe with averages of base pay, benefits, and overtime for each job title, as well as a column with the average of these three values. Use `JobTitle`, `AvgBasePay`, `AvgBenefits`, `AvgOvertimePay`, `AvgTotal` as the column name respectively.

In [9]:
query = 'SELECT JobTitle, AVG(BasePay) AS AvgBasePay, AVG(Benefits) AS AvgBenefits, AVG(OvertimePay) AS AvgOvertimePay, (AVG(BasePay) + AVG(Benefits) + AVG(OvertimePay)) / 3 AS AvgTotal FROM salaries GROUP BY JobTitle;'
df6 = pd.read_sql(query, conn)
df6.head()


,JobTitle,AvgBasePay,AvgBenefits,AvgOvertimePay,AvgTotal
0,ACCOUNT CLERK,43300.806506,NaN,373.200843,NaN
1,ACCOUNTANT,46643.172000,NaN,0.000000,NaN
2,ACCOUNTANT INTERN,28732.663958,NaN,24.430625,NaN
3,"ACPO,JuvP, Juv Prob (SFERS)",62290.780000,17975.59,0.000000,26755.456667
4,ACUPUNCTURIST,66374.400000,NaN,0.000000,NaN


# Table Creation


Now we'll create our own table in our database. Separate the Salaries table by Year, and add it back to the database.

Use `Y` + `Year` as the table name (i.e. `Y2011`). 

Use `df7_list` as a list variable of dataframe to save the data. For example, `df7_list[0]` will represent Year 2011's data.


In [10]:
df7_list = []
for y in ['2011','2012','2013','2014']:
    query = f'SELECT * FROM salaries WHERE Year = {y} ;'
    df7 = pd.read_sql(query, conn)
    df7.to_sql(name='Y'+y, con=conn, if_exists='replace')
    df7_list.append(df7)

df7_list[0].head()

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,None,567595.43,567595.43,2011,None,San Francisco,None
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,None,538909.28,538909.28,2011,None,San Francisco,None
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,None,335279.91,335279.91,2011,None,San Francisco,None
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,None,332343.61,332343.61,2011,None,San Francisco,None
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,None,326373.19,326373.19,2011,None,San Francisco,None


# Table Joining

Problem 8:

We'll move on to a new dataset for the next steps.
Download the dataset from here (https://www.kaggle.com/datasets/luizpaulodeoliveira/imdb-project-sql) and load the sqlite file. Select everything in the "movies" table to see what it looks like. Use `df8` as the pandas DataFrame variable to store your query result

In [11]:
conn = sqlite3.connect("movies.sqlite")
query = 'SELECT * FROM movies'
df8 = pd.read_sql(query, conn)
df8.head()

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766



Create a dataframe that includes the entire contents of "movies" table as well as the director's name. Use `director_name` as the column name of the directore's name. 

In [12]:
# check the schema of the directors table
query = "PRAGMA table_info(directors)"
df = pd.read_sql(query, conn)
print(df.head())

query = 'SELECT *,directors.name AS director_name FROM movies LEFT JOIN directors ON movies.director_id = directors.id;'
df9 = pd.read_sql(query, conn)
df9.head()

   cid        name     type  notnull dflt_value  pk
0    0        name     TEXT        0       None   0
1    1          id  INTEGER        0       None   1
2    2      gender  INTEGER        0       None   0
3    3         uid  INTEGER        0       None   0
4    4  department     TEXT        0       None   0


,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id,name,id,gender,uid,department,director_name
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762,James Cameron,4762,2,2710,Directing,James Cameron
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763,Gore Verbinski,4763,2,1704,Directing,Gore Verbinski
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764,Sam Mendes,4764,2,39,Directing,Sam Mendes
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765,Christopher Nolan,4765,2,525,Directing,Christopher Nolan
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766,Andrew Stanton,4766,2,7,Directing,Andrew Stanton


# Analysis

What is the average budget used for the top 10 grossing movies? Result should have a column `avg_budget` for the average budget used for the top 10 grossing movies. 

In [13]:
query = """ SELECT AVG(budget) AS avg_budget
    FROM (
    SELECT budget
    FROM movies
    ORDER BY revenue DESC
    LIMIT 10) AS top10;"""
    
df10 = pd.read_sql(query, conn)
df10.head()

,avg_budget
0,195100000.0



Which directors have the highest overall voting average? - show the top 5 directors' name and their average rating. Your result should have a column `ave_rating` and a column `director_name`.

In [14]:
#Check these for correctness
query = 'SELECT directors.name AS director_name, AVG(vote_average) AS avg_rating FROM movies LEFT JOIN directors ON movies.director_id = directors.id GROUP BY director_name ORDER BY avg_rating DESC LIMIT 5;'
df11 = pd.read_sql(query, conn)
df11.head()


,director_name,avg_rating
0,Gary Sinyor,10.00
1,Rohit Jugraj,9.50
2,Lance Hool,9.30
3,Floyd Mutrux,8.50
4,Tim McCanlies,8.45


What are the top five directors by how much their average budget is? Your result should have a column `ave_budget` and a column `director_name`. 

In [15]:
# Problem 12: Top 5 directors by average budget
#check these for correctness
query = 'SELECT directors.name AS director_name, AVG(budget) AS avg_budget FROM movies LEFT JOIN directors ON movies.director_id = directors.id GROUP BY director_name ORDER BY avg_budget DESC LIMIT 5;'
df12 = pd.read_sql(query, conn)
df12.head()

,director_name,avg_budget
0,Byron Howard,2.600000e+08
1,Lee Unkrich,2.000000e+08
2,Dan Scanlon,2.000000e+08
3,David Yates,1.933333e+08
4,Brenda Chapman,1.850000e+08
